In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tifffile as tiff
import matplotlib.pyplot as plt

# HANDING FUNCTIONS

In [ ]:
## We need to decode the mask from encoding column of train.csv
## https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    #print(starts, ends)
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
# Simple DICE Coefficient Implementation
def DICE_COEFF(mask1, mask2):
    intersect = np.sum(mask1*mask2)
    sum1 = np.sum(mask1)
    sum2 = np.sum(mask2)
    dice = 2*intersect/(sum1+sum2)
    dice = np.mean(dice)
    return round(dice, 3)

In [ ]:
# Shifting the images towards the bottom to keep it simple
def return_shifted(mask, shift=5):
    nmask = np.zeros((mask.shape[0]+shift, mask.shape[1]))
    nmask[shift:, :] = mask
    nmask = nmask[:-shift, :]
    return nmask

# Loading

In [ ]:
TRAIN_PATH = "../input/hubmap-organ-segmentation/train_images"
TEST_PATH = "../input/hubmap-organ-segmentation/test_images"

# Training Dataset Information
train_df = pd.read_csv("../input/hubmap-organ-segmentation/train.csv")
print(f"Shape of the Training Dataset : {train_df.shape}")

In [ ]:
train_df.head()

# Image and the Mask

In [ ]:
img1 = tiff.imread(TRAIN_PATH +'/' + str(train_df.id[4]) + '.tiff')
img1.shape

In [ ]:
plt.imshow(img1)

In [ ]:
rle = train_df.rle[4]

In [ ]:
mask = rle2mask(rle, img1.shape[:2])

# Check with mask and mask
Yes! You can obtain the answer by easy calculation

Let the number of masked pixels be x . Then the intersection between the two images will also have x pixels.

$\huge \frac{2\cdot x }{x + x}  = 1.0$

In [ ]:
DICE_COEFF(mask, mask)

# Check with Shifted Image

In [ ]:
sh_mask = return_shifted(mask)

In [ ]:
DICE_COEFF(mask, sh_mask)

# To be continued